In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
from google.colab import drive
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from pandas.plotting import table

# Configurações
drive.mount('/content/drive')
GRAPHPATH = "/content/drive/MyDrive/graficos"
SUMMARYPATH = "/content/drive/MyDrive/sumarios"
CSV_URL = "https://docs.google.com/spreadsheets/d/1P9kPXPfW765EmXwFHNnfCczYIz7Qsr4KSZKYgovCf5g/export?format=csv&gid=1597010775"

# Estilo dos gráficos
sns.set(style="whitegrid")
plt.rcParams.update({
    'figure.facecolor': 'black', 'axes.facecolor': 'black', 'axes.edgecolor': 'white',
    'xtick.color': 'white', 'ytick.color': 'white', 'text.color': 'white',
    'legend.facecolor': 'black'
})

# Funções auxiliares para salvar gráficos
def save_fig(filename):
    plt.savefig(os.path.join(GRAPHPATH, filename), dpi=300, bbox_inches='tight')
    plt.close()

# Carregar dados
def load_data(url):
    dtype = {'EEG_alfa': 'float', 'EEG_Beta': 'float', 'ECG_Freq': 'int', 'EDA_Cond': 'float',
             'Eye_Fix': 'int', 'Uso_de_LLMs': 'int', 'Administracao_de_Fitoterapicos': 'int'}
    return pd.read_csv(url, dtype=dtype)

# Função para gerar gráficos de violino
def generate_violinplot(df, y, x, hue, title, filename):
    plt.figure(figsize=(10, 6))
    sns.violinplot(data=df, y=y, x=x, hue=hue, split=True, inner="quart", palette="muted")
    plt.title(title)
    save_fig(filename)

# Função para gerar gráficos KDE
def generate_kdeplot(df, x, hue, title, filename):
    plt.figure(figsize=(10, 6))
    sns.kdeplot(data=df, x=x, hue=hue, fill=True, common_norm=False, palette="muted", alpha=0.7)
    plt.title(title)
    save_fig(filename)

# Função para gerar histogramas
def generate_histplot(df, x, hue, title, filename):
    plt.figure(figsize=(10, 6))
    sns.histplot(data=df, x=x, hue=hue, multiple="stack", palette="muted", kde=True)
    plt.title(title)
    save_fig(filename)

# Função para gerar gráficos de boxplot
def generate_boxplot(df, y, x, title, filename):
    plt.figure(figsize=(10, 6))
    sns.boxplot(data=df, y=y, x=x, palette="muted")
    plt.title(title)
    save_fig(filename)

# Função para gerar regressão polinomial
def generate_polynomial_regression(df, x, y, title, filename, degree=2):
    plt.figure(figsize=(10, 6))
    poly = PolynomialFeatures(degree=degree)
    X_poly = poly.fit_transform(df[[x]])
    model = LinearRegression().fit(X_poly, df[y])
    X_range = np.linspace(df[x].min(), df[x].max(), 100).reshape(-1, 1)
    y_pred = model.predict(poly.transform(X_range))
    sns.scatterplot(data=df, x=x, y=y, color="blue", alpha=0.6, label="Dados")
    plt.plot(X_range, y_pred, color="red", label=f'Regressão Polinomial (grau {degree})')
    plt.title(title)
    plt.legend()
    save_fig(filename)

# Função para gerar sumário estatístico simplificado
def generate_statistical_summary(df):
    summary = df.describe()

    # Salvar sumário estatístico
    with open(os.path.join(SUMMARYPATH, 'sumario_estatistico_simplificado.txt'), 'w') as f:
        f.write(summary.to_string())

    # Salvar sumário como imagem
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.axis('tight')
    ax.axis('off')
    table(ax, summary, loc='center', cellLoc='center', colWidths=[0.1] * len(summary.columns))
    plt.savefig(os.path.join(SUMMARYPATH, 'sumario_estatistico_simplificado.png'), dpi=300, bbox_inches='tight')
    plt.close()

# Criar pastas para salvar gráficos e sumários
os.makedirs(GRAPHPATH, exist_ok=True)
os.makedirs(SUMMARYPATH, exist_ok=True)

# Carregar dados
df = load_data(CSV_URL)

# Gerar sumário estatístico
generate_statistical_summary(df)

# Gerar gráficos
violin_plots = [
    ('EEG_alfa', 'Uso_de_LLMs', 'Administracao_de_Fitoterapicos',
     'Violin Plot: EEG Alfa por Uso de LLMs e Fitoterápicos', 'violin_eeg_alfa_llms_fitoterapicos.png'),
    ('EEG_Beta', 'Uso_de_LLMs', 'Administracao_de_Fitoterapicos',
     'Violin Plot: EEG Beta por Uso de LLMs e Fitoterápicos', 'violin_eeg_beta_llms_fitoterapicos.png'),
    ('ECG_Freq', 'Uso_de_LLMs', 'Administracao_de_Fitoterapicos',
     'Violin Plot: ECG Frequência por Uso de LLMs e Fitoterápicos', 'violin_ecg_llms_fitoterapicos.png'),
    ('EDA_Cond', 'Uso_de_LLMs', 'Administracao_de_Fitoterapicos',
     'Violin Plot: EDA Condutância por Uso de LLMs e Fitoterápicos', 'violin_eda_llms_fitoterapicos.png'),
    ('EEG_alfa', 'Eye_Fix', 'Administracao_de_Fitoterapicos',
     'Violin Plot: EEG Alfa por Eye Fix e Fitoterápicos', 'violin_eeg_alfa_eye_fix_fitoterapicos.png'),
    ('EEG_Beta', 'Eye_Fix', 'Administracao_de_Fitoterapicos',
     'Violin Plot: EEG Beta por Eye Fix e Fitoterápicos', 'violin_eeg_beta_eye_fix_fitoterapicos.png'),
    ('ECG_Freq', 'Eye_Fix', 'Administracao_de_Fitoterapicos',
     'Violin Plot: ECG Frequência por Eye Fix e Fitoterápicos', 'violin_ecg_eye_fix_fitoterapicos.png'),
    ('EDA_Cond', 'Eye_Fix', 'Administracao_de_Fitoterapicos',
     'Violin Plot: EDA Condutância por Eye Fix e Fitoterápicos', 'violin_eda_eye_fix_fitoterapicos.png')
]

kde_plots = [
    ('EEG_alfa', 'Uso_de_LLMs', 'KDE: EEG Alfa por Uso de LLMs', 'kde_eeg_alfa_llms.png'),
    ('EEG_Beta', 'Uso_de_LLMs', 'KDE: EEG Beta por Uso de LLMs', 'kde_eeg_beta_llms.png'),
    ('ECG_Freq', 'Uso_de_LLMs', 'KDE: Frequência Cardíaca (ECG) por Uso de LLMs', 'kde_ecg_llms.png'),
    ('EDA_Cond', 'Uso_de_LLMs', 'KDE: Condutância Eletrodérmica (EDA) por Uso de LLMs', 'kde_eda_llms.png'),
    ('EEG_alfa', 'Administracao_de_Fitoterapicos', 'KDE: EEG Alfa por Fitoterápicos', 'kde_eeg_alfa_fitoterapicos.png'),
    ('EEG_Beta', 'Administracao_de_Fitoterapicos', 'KDE: EEG Beta por Fitoterápicos', 'kde_eeg_beta_fitoterapicos.png'),
    ('ECG_Freq', 'Administracao_de_Fitoterapicos', 'KDE: Frequência Cardíaca (ECG) por Fitoterápicos', 'kde_ecg_fitoterapicos.png'),
    ('EDA_Cond', 'Administracao_de_Fitoterapicos', 'KDE: Condutância Eletrodérmica (EDA) por Fitoterápicos', 'kde_eda_fitoterapicos.png')
]

hist_plots = [
    ('EEG_alfa', 'Uso_de_LLMs', 'Histograma: EEG Alfa por Uso de LLMs', 'histograma_eeg_alfa_llms.png'),
    ('EEG_Beta', 'Uso_de_LLMs', 'Histograma: EEG Beta por Uso de LLMs', 'histograma_eeg_beta_llms.png'),
    ('ECG_Freq', 'Uso_de_LLMs', 'Histograma: Frequência Cardíaca (ECG) por Uso de LLMs', 'histograma_ecg_llms.png'),
    ('EDA_Cond', 'Uso_de_LLMs', 'Histograma: Condutância Eletrodérmica (EDA) por Uso de LLMs', 'histograma_eda_llms.png')
]

box_plots = [
    ('EEG_alfa', 'Administracao_de_Fitoterapicos', 'Boxplot: EEG Alfa por Fitoterápicos', 'boxplot_eeg_alfa_fitoterapicos.png'),
    ('EEG_Beta', 'Administracao_de_Fitoterapicos', 'Boxplot: EEG Beta por Fitoterápicos', 'boxplot_eeg_beta_fitoterapicos.png'),
    ('ECG_Freq', 'Administracao_de_Fitoterapicos', 'Boxplot: Frequência Cardíaca por Fitoterápicos', 'boxplot_ecg_fitoterapicos.png'),
    ('EDA_Cond', 'Administracao_de_Fitoterapicos', 'Boxplot: Condutância Eletrodérmica por Fitoterápicos', 'boxplot_eda_fitoterapicos.png')
]

polynomial_regressions = [
    ('EEG_alfa', 'Uso_de_LLMs', 'Regressão Polinomial: EEG Alfa por Uso de LLMs', 'regressao_eeg_alfa_llms.png'),
    ('EEG_Beta', 'Uso_de_LLMs', 'Regressão Polinomial: EEG Beta por Uso de LLMs', 'regressao_eeg_beta_llms.png'),
    ('ECG_Freq', 'Uso_de_LLMs', 'Regressão Polinomial: Frequência Cardíaca por Uso de LLMs', 'regressao_ecg_llms.png'),
    ('EDA_Cond', 'Uso_de_LLMs', 'Regressão Polinomial: Condutância Eletrodérmica por Uso de LLMs', 'regressao_eda_llms.png'),
    ('EEG_alfa', 'Administracao_de_Fitoterapicos', 'Regressão Polinomial: EEG Alfa por Fitoterápicos', 'regressao_eeg_alfa_fitoterapicos.png'),
    ('EEG_Beta', 'Administracao_de_Fitoterapicos', 'Regressão Polinomial: EEG Beta por Fitoterápicos', 'regressao_eeg_beta_fitoterapicos.png'),
    ('ECG_Freq', 'Administracao_de_Fitoterapicos', 'Regressão Polinomial: Frequência Cardíaca por Fitoterápicos', 'regressao_ecg_fitoterapicos.png'),
    ('EDA_Cond', 'Administracao_de_Fitoterapicos', 'Regressão Polinomial: Condutância Eletrodérmica por Fitoterápicos', 'regressao_eda_fitoterapicos.png')
]

# Gerar e salvar gráficos de violino
for y, x, hue, title, filename in violin_plots:
    generate_violinplot(df, y, x, hue, title, filename)

# Gerar e salvar gráficos KDE
for x, hue, title, filename in kde_plots:
    generate_kdeplot(df, x, hue, title, filename)

# Gerar e salvar histogramas
for x, hue, title, filename in hist_plots:
    generate_histplot(df, x, hue, title, filename)

# Gerar e salvar boxplots
for y, x, title, filename in box_plots:
    generate_boxplot(df, y, x, title, filename)

# Gerar e salvar gráficos de regressão polinomial
for x, y, title, filename in polynomial_regressions:
    generate_polynomial_regression(df, x, y, title, filename)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-20-d5e0212746c2>:60: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(data=df, y=y, x=x, palette="muted")
<ipython-input-20-d5e0212746c2>:60: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(data=df, y=y, x=x, palette="muted")
<ipython-input-20-d5e0212746c2>:60: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(data=df, y=y, x=x, palette="muted")
<ipython-input-20-d5e0212746c2>:60: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=Fal